# notebook for the capstone

### load/install librarys

In [2]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json

from sklearn.cluster import KMeans
!pip install pyproj
from pyproj import Transformer

!pip install geopandas
import geopandas as gpd  # need to build a json with WGS format, folium use that format in geojson
!pip install folium
import folium

     |████████████████████████████████| 10.9MB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 931kB 14.3MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 36.6MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 57.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 8.1MB/s eta 0:00:011


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### data collection

In [4]:
!wget -q -O 'valencia_data.json' http://mapas.valencia.es/lanzadera/opendata/Barrios/JSON
print('Data downloaded!')

Data downloaded!


In [5]:
with open('valencia_data.json') as json_data:
    valencia_data = json.load(json_data)

In [136]:
neighborhoods_data = valencia_data['features']
# define the dataframe columns
column_names = ['coddistbar', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
transformer = Transformer.from_crs("epsg:25830", "epsg:4326")  # convert utm30 to wgs84

for data in neighborhoods_data:
    coddistbar = data['properties']['coddistbar'] 
    # coddistbar = neighborhood_name = data['properties']['coddistbar'] 
    neighborhood_name = data['properties']['nombre']
        
    neighborhood_latlon = np.array(data['geometry']['coordinates'][0])
    
    #neighborhood_lat = neighborhood_latlon.mean(axis=0)[1]
    #neighborhood_lon = neighborhood_latlon.mean(axis=0)[0]
    neighborhood_lat, neighborhood_lon = transformer.transform(neighborhood_latlon.mean(axis=0)[0], neighborhood_latlon.mean(axis=0)[1])
    neighborhoods = neighborhoods.append({'coddistbar': coddistbar,
                                          'Neighborhood': neighborhood_name,
                                          'x': neighborhood_latlon.mean(axis=0)[0],
                                          'y': neighborhood_latlon.mean(axis=0)[1],
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [137]:
neighborhoods.style.format("{:.4f}")
pd.options.display.float_format = '{:.5f}'.format
neighborhoods.head()


,coddistbar,Neighborhood,Latitude,Longitude,x,y
0,171,BENIFARAIG,39.52843,-0.38583,724687.45042,4378683.63279
1,161,BENICALAP,39.49260,-0.39022,724425.06294,4374694.98757
2,152,TORREFIEL,39.49518,-0.37592,725646.59555,4375017.44694
3,054,TORMOS,39.48895,-0.37955,725354.34755,4374316.40432
4,055,SANT ANTONI,39.48838,-0.37355,725872.71353,4374268.68979


In [8]:
df_airbnb = pd.read_csv('https://code.montera34.com/airbnb/valencia/-/raw/master/data/original/airbnb/190227/listings_summary_valencia_insideairbnb.csv', index_col=False)

In [9]:
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,4620,VALENCIA HISTORIC HOUSE 50M BEACH,7093832,Francisca,POBLATS MARITIMS,LA MALVA-ROSA,39.47553,-0.32461,Entire home/apt,150,2,76,2019-01-21,0.91000,1,305
1,45528,Valencia: El Saler Beach and Albufera Lake,202104,Lidia,POBLATS DEL SUD,EL SALER,39.35621,-0.32292,Entire home/apt,60,3,46,2018-12-08,0.65000,1,50
2,48154,beautiful apartment wifi,219476,Toni,LA SAIDIA,MORVEDRE,39.48575,-0.37428,Entire home/apt,45,3,71,2018-12-06,0.70000,3,345
3,55692,Delight in city center,120649,Pilar,CIUTAT VELLA,LA SEU,39.47681,-0.37523,Entire home/apt,100,1,1,2014-11-11,0.02000,1,146
4,95393,Modern and bright Duplex 3 Bedrooms Center Wi-Fi,508042,Carmen,CIUTAT VELLA,EL CARME,39.47787,-0.38142,Entire home/apt,134,1,88,2019-02-22,0.94000,6,167


In [10]:
np.sort(neighborhoods['Neighborhood'].unique())

array(['AIORA', 'ALBORS', 'ARRANCAPINS', 'BENICALAP', 'BENIFARAIG',
       'BENIFERRI', 'BENIMACLET', 'BENIMAMET', 'BETERO', 'BORBOTO',
       'CABANYAL-CANYAMELAR', 'CAMI DE VERA', 'CAMI FONDO', 'CAMI REAL',
       'CAMPANAR', 'CARPESA', "CASTELLAR-L'OLIVERAL",
       'CIUTAT DE LES ARTS I DE LES CIENCIES', 'CIUTAT FALLERA',
       'CIUTAT JARDI', 'CIUTAT UNIVERSITARIA', 'EL BOTANIC', 'EL CALVARI',
       'EL CARME', "EL FORN D'ALCEDO", 'EL GRAU', 'EL MERCAT',
       'EL PALMAR', 'EL PERELLONET', 'EL PILAR', 'EL PLA DEL REMEI',
       'EL SALER', 'ELS ORRIOLS', 'EN CORTS', 'EXPOSICIO', 'FAITANAR',
       'FAVARA', 'JAUME ROIG', "L'AMISTAT", "L'HORT DE SENABRE",
       "L'ILLA PERDUDA", 'LA CARRASCA', 'LA CREU COBERTA',
       'LA CREU DEL GRAU', 'LA FONTETA S.LLUIS', 'LA FONTSANTA',
       'LA GRAN VIA', 'LA LLUM', 'LA MALVA-ROSA', 'LA PETXINA',
       'LA PUNTA', 'LA RAIOSA', 'LA ROQUETA', 'LA SEU', 'LA TORRE',
       'LA VEGA BAIXA', 'LA XEREA', 'LES CASES DE BARCENA',
       'LES T

In [138]:
df = df_airbnb.groupby('neighbourhood')['price'].mean().to_frame().reset_index().rename(columns={'neighbourhood': 'Neighborhood'})
df_std = df_airbnb.groupby('neighbourhood').std()[['price']].reset_index().rename(columns={'price':'price_std', 'neighbourhood': 'Neighborhood'})
df_count = df_airbnb.groupby('neighbourhood').count()[['price']].reset_index().rename(columns={'price':'number', 'neighbourhood': 'Neighborhood'})
df = df.merge(df_std, on=['Neighborhood']).fillna(0)
df = df.merge(df_count, on=['Neighborhood'])

In [90]:
df.count()

Neighborhood    87
price           87
price_std       87
number          87
dtype: int64

In [155]:
df1 = df.merge(neighborhoods, on='Neighborhood')
df1 = df1.astype({'coddistbar':'int'})
df1.head(25)


Neighborhood     object
price           float64
price_std       float64
number            int64
coddistbar        int64
Latitude        float64
Longitude       float64
x               float64
y               float64
dtype: object

In [47]:
df1.shape

(87, 9)

In [48]:
neighborhoods.shape

(88, 6)

In [61]:
set(df1.Neighborhood.to_list()).symmetric_difference(neighborhoods.Neighborhood)

{'BENIFARAIG'}

In [140]:
data = gpd.read_file("valencia_data.json")
data.to_crs(epsg=4326).to_file("valencia_data_wgs.json", driver="GeoJSON")
m = folium.Map(location=[39.46, -.3], zoom_start=11)
folium.GeoJson(
    "valencia_data_wgs.json",
    name='geojson'
).add_to(m)


m

In [215]:
m = folium.Map(location=[39.46, -.3], zoom_start=11)
bins = list(df1['price'].quantile([0, 0.1, 0.6, 0.85, 1]))
folium.Choropleth(
    geo_data="valencia_data_wgs.json",
    name='choropleth',
    data=df1,
    columns=['Neighborhood', 'price'],
    key_on='feature.properties.nombre',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins=bins,
    legend_name='price of houses for rent'
).add_to(m)




m

In [202]:
df_mapa.isna().any().any()

False

In [205]:
m = folium.Map(location=[39.46, -.3], zoom_start=11)
folium.Choropleth(
    geo_data="valencia_data_wgs.json",
    name='choropleth',
    data=df1,
    columns=['Neighborhood', 'number'],
    key_on='feature.properties.nombre',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='number of houses for rent'
).add_to(m)

m

In [125]:
# The code was removed by Watson Studio for sharing.

In [149]:
neighborhood_latitude = df1.loc[9, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df1.loc[9, 'Longitude'] # neighborhood longitude value
LIMIT = 100
radius = 500
VERSION = '20180604'
#VERSION = '20190604'

In [150]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)

In [151]:
results = requests.get(url).json()
#print('There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items'])))
results

{'meta': {'code': 200, 'requestId': '5eb14cf769babe001ba2390a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'El Cabanyal-El Canyamelar',
  'headerFullLocation': 'El Cabanyal-El Canyamelar, Valencia',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 27,
  'suggestedBounds': {'ne': {'lat': 39.47420887198479,
    'lng': -0.32505476474291856},
   'sw': {'lat': 39.46520886298479, 'lng': -0.33669163211944036}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d0cb25e903d3704ddf0be55',
       'name': 'Bar Bodega "Flor"',
       'location': {'address': 'Carrer Martí Grajales 21',
        'crossStreet': 'Carrer Luis Despuig',
        'lat': 39.46853455789225,
        'ln

In [129]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [152]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,  
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
valencia_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude'],
                                  radius=radius
                                  )


NameError: name 'getNearbyVenues' is not defined

In [155]:
valencia_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIORA,39.46619,-0.34449,Bar Mochuelos,39.46614,-0.34794,Mediterranean Restaurant
1,AIORA,39.46619,-0.34449,McFIT,39.46691,-0.35119,Gym / Fitness Center
2,AIORA,39.46619,-0.34449,Jardines de Ayora,39.46814,-0.34340,Playground
3,AIORA,39.46619,-0.34449,Taco A Saco,39.46782,-0.34775,Mexican Restaurant
4,AIORA,39.46619,-0.34449,Café Infinito,39.47063,-0.34832,Vegetarian / Vegan Restaurant


In [169]:
# one hot encoding
valencia_onehot = pd.get_dummies(valencia_venues[['Venue Category']], prefix="", prefix_sep="")

fixed_columns = ['Neighborhood'] + list(set(valencia_onehot.columns.to_list()).difference(['Neighborhood']))
# add neighborhood column back to dataframe
valencia_onehot['Neighborhood'] = valencia_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [valencia_onehot.columns[-1]] + list(valencia_onehot.columns[:-1])

valencia_onehot = valencia_onehot[fixed_columns]

valencia_onehot.head()

,Neighborhood,Arts & Crafts Store,Pool,Buffet,Tapas Restaurant,Photography Studio,Spanish Restaurant,General Entertainment,Harbor / Marina,Hockey Field,Brewery,Molecular Gastronomy Restaurant,Candy Store,Liquor Store,Boutique,Art Museum,Aquarium,Beer Bar,Gym / Fitness Center,Cheese Shop,Light Rail Station,Noodle House,Japanese Restaurant,Dog Run,Church,Dessert Shop,Music School,Historic Site,Shop & Service,College Cafeteria,Irish Pub,Italian Restaurant,Comic Shop,Public Art,Performing Arts Venue,Theater,BBQ Joint,Garden Center,Furniture / Home Store,Southern / Soul Food Restaurant,Dive Bar,Planetarium,Pub,Post Office,Sporting Goods Shop,Jazz Club,Warehouse Store,Pet Store,Basketball Stadium,Sports Club,Arepa Restaurant,Coffee Shop,Zoo Exhibit,Modern European Restaurant,Restaurant,Bed & Breakfast,Botanical Garden,Garden,History Museum,Music Store,Seafood Restaurant,Soccer Field,Toy / Game Store,Martial Arts Dojo,Fast Food Restaurant,Brazilian Restaurant,Beach,Nudist Beach,College Stadium,Cuban Restaurant,Grocery Store,Food Court,Playground,Museum,Music Venue,Bridge,Beer Garden,Department Store,Mobile Phone Shop,Cruise,Supermarket,Theme Park,Comedy Club,Farmers Market,Electronics Store,Indie Movie Theater,Theme Restaurant,Zoo,Food Truck,Opera House,Gift Shop,Bookstore,Hotel Bar,Greek Restaurant,Rest Area,Clothing Store,Pizza Place,Outdoors & Recreation,Nature Preserve,Video Store,Movie Theater,Scenic Lookout,Camera Store,Deli / Bodega,Strip Club,Shoe Store,Bistro,Golf Course,Moroccan Restaurant,Other Great Outdoors,Surf Spot,Other Nightlife,Football Stadium,Middle Eastern Restaurant,Convention Center,Snack Place,Gourmet Shop,Taco Place,Big Box Store,Pastry Shop,Road,Multiplex,Gas Station,Asian Restaurant,Rock Club,Shopping Mall,Stadium,Thrift / Vintage Store,Trail,Speakeasy,Track Stadium,Food,Falafel Restaurant,Hotel,Beer Store,Mediterranean Restaurant,Racetrack,Video Game Store,Cupcake Shop,American Restaurant,Event Space,Gaming Cafe,Latin American Restaurant,Convenience Store,German Restaurant,Café,Argentinian Restaurant,Bakery,Salad Place,Miscellaneous Shop,Exhibit,Market,Monument / Landmark,Indian Restaurant,Food & Drink Shop,Juice Bar,Record Shop,Bagel Shop,Tea Room,Sushi Restaurant,Ramen Restaurant,Tennis Court,Beach Bar,Tennis Stadium,Boat or Ferry,Pharmacy,Cosmetics Shop,Creperie,Gym,Hostel,Plaza,Wine Bar,Art Gallery,Cocktail Bar,Mexican Restaurant,Bike Rental / Bike Share,Metro Station,Modern Greek Restaurant,Soccer Stadium,Paella Restaurant,French Restaurant,Frozen Yogurt Shop,Pedestrian Plaza,Korean Restaurant,Tourist Information Center,Burger Joint,Lingerie Store,Halal Restaurant,Gym Pool,Casino,Concert Hall,Spa,Steakhouse,Diner,Breakfast Spot,Ice Cream Shop,Pie Shop,Sandwich Place,Science Museum,Lounge,Soup Place,Gastropub,Nightclub,Bowling Alley,Bar,Train Station,Athletics & Sports,Park,Vegetarian / Vegan Restaurant,Sports Bar,Forest,Chinese Restaurant
0,AIORA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AIORA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AIORA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [170]:
valencia_onehot = valencia_onehot.groupby('Neighborhood').mean().reset_index()
valencia_onehot

,Neighborhood,Arts & Crafts Store,Pool,Buffet,Tapas Restaurant,Photography Studio,Spanish Restaurant,General Entertainment,Harbor / Marina,Hockey Field,Brewery,Molecular Gastronomy Restaurant,Candy Store,Liquor Store,Boutique,Art Museum,Aquarium,Beer Bar,Gym / Fitness Center,Cheese Shop,Light Rail Station,Noodle House,Japanese Restaurant,Dog Run,Church,Dessert Shop,Music School,Historic Site,Shop & Service,College Cafeteria,Irish Pub,Italian Restaurant,Comic Shop,Public Art,Performing Arts Venue,Theater,BBQ Joint,Garden Center,Furniture / Home Store,Southern / Soul Food Restaurant,Dive Bar,Planetarium,Pub,Post Office,Sporting Goods Shop,Jazz Club,Warehouse Store,Pet Store,Basketball Stadium,Sports Club,Arepa Restaurant,Coffee Shop,Zoo Exhibit,Modern European Restaurant,Restaurant,Bed & Breakfast,Botanical Garden,Garden,History Museum,Music Store,Seafood Restaurant,Soccer Field,Toy / Game Store,Martial Arts Dojo,Fast Food Restaurant,Brazilian Restaurant,Beach,Nudist Beach,College Stadium,Cuban Restaurant,Grocery Store,Food Court,Playground,Museum,Music Venue,Bridge,Beer Garden,Department Store,Mobile Phone Shop,Cruise,Supermarket,Theme Park,Comedy Club,Farmers Market,Electronics Store,Indie Movie Theater,Theme Restaurant,Zoo,Food Truck,Opera House,Gift Shop,Bookstore,Hotel Bar,Greek Restaurant,Rest Area,Clothing Store,Pizza Place,Outdoors & Recreation,Nature Preserve,Video Store,Movie Theater,Scenic Lookout,Camera Store,Deli / Bodega,Strip Club,Shoe Store,Bistro,Golf Course,Moroccan Restaurant,Other Great Outdoors,Surf Spot,Other Nightlife,Football Stadium,Middle Eastern Restaurant,Convention Center,Snack Place,Gourmet Shop,Taco Place,Big Box Store,Pastry Shop,Road,Multiplex,Gas Station,Asian Restaurant,Rock Club,Shopping Mall,Stadium,Thrift / Vintage Store,Trail,Speakeasy,Track Stadium,Food,Falafel Restaurant,Hotel,Beer Store,Mediterranean Restaurant,Racetrack,Video Game Store,Cupcake Shop,American Restaurant,Event Space,Gaming Cafe,Latin American Restaurant,Convenience Store,German Restaurant,Café,Argentinian Restaurant,Bakery,Salad Place,Miscellaneous Shop,Exhibit,Market,Monument / Landmark,Indian Restaurant,Food & Drink Shop,Juice Bar,Record Shop,Bagel Shop,Tea Room,Sushi Restaurant,Ramen Restaurant,Tennis Court,Beach Bar,Tennis Stadium,Boat or Ferry,Pharmacy,Cosmetics Shop,Creperie,Gym,Hostel,Plaza,Wine Bar,Art Gallery,Cocktail Bar,Mexican Restaurant,Bike Rental / Bike Share,Metro Station,Modern Greek Restaurant,Soccer Stadium,Paella Restaurant,French Restaurant,Frozen Yogurt Shop,Pedestrian Plaza,Korean Restaurant,Tourist Information Center,Burger Joint,Lingerie Store,Halal Restaurant,Gym Pool,Casino,Concert Hall,Spa,Steakhouse,Diner,Breakfast Spot,Ice Cream Shop,Pie Shop,Sandwich Place,Science Museum,Lounge,Soup Place,Gastropub,Nightclub,Bowling Alley,Bar,Train Station,Athletics & Sports,Park,Vegetarian / Vegan Restaurant,Sports Bar,Forest,Chinese Restaurant
0,AIORA,0.00000,0.00000,0.00000,0.10000,0.00000,0.05000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.06000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.07000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.01000,0.00000,0.00000,0.03000,0.00000,0.00000,0.00000,0.00000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.01000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.01000,0.00000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.04000,0.00000,0.0

In [173]:
df2 = df1.merge(valencia_onehot, on='Neighborhood')
df2.head()

,Neighborhood,price,Borough,Latitude,Longitude,x,y,Arts & Crafts Store,Pool,Buffet,Tapas Restaurant,Photography Studio,Spanish Restaurant,General Entertainment,Harbor / Marina,Hockey Field,Brewery,Molecular Gastronomy Restaurant,Candy Store,Liquor Store,Boutique,Art Museum,Aquarium,Beer Bar,Gym / Fitness Center,Cheese Shop,Light Rail Station,Noodle House,Japanese Restaurant,Dog Run,Church,Dessert Shop,Music School,Historic Site,Shop & Service,College Cafeteria,Irish Pub,Italian Restaurant,Comic Shop,Public Art,Performing Arts Venue,Theater,BBQ Joint,Garden Center,Furniture / Home Store,Southern / Soul Food Restaurant,Dive Bar,Planetarium,Pub,Post Office,Sporting Goods Shop,Jazz Club,Warehouse Store,Pet Store,Basketball Stadium,Sports Club,Arepa Restaurant,Coffee Shop,Zoo Exhibit,Modern European Restaurant,Restaurant,Bed & Breakfast,Botanical Garden,Garden,History Museum,Music Store,Seafood Restaurant,Soccer Field,Toy / Game Store,Martial Arts Dojo,Fast Food Restaurant,Brazilian Restaurant,Beach,Nudist Beach,College Stadium,Cuban Restaurant,Grocery Store,Food Court,Playground,Museum,Music Venue,Bridge,Beer Garden,Department Store,Mobile Phone Shop,Cruise,Supermarket,Theme Park,Comedy Club,Farmers Market,Electronics Store,Indie Movie Theater,Theme Restaurant,Zoo,Food Truck,Opera House,Gift Shop,Bookstore,Hotel Bar,Greek Restaurant,Rest Area,Clothing Store,Pizza Place,Outdoors & Recreation,Nature Preserve,Video Store,Movie Theater,Scenic Lookout,Camera Store,Deli / Bodega,Strip Club,Shoe Store,Bistro,Golf Course,Moroccan Restaurant,Other Great Outdoors,Surf Spot,Other Nightlife,Football Stadium,Middle Eastern Restaurant,Convention Center,Snack Place,Gourmet Shop,Taco Place,Big Box Store,Pastry Shop,Road,Multiplex,Gas Station,Asian Restaurant,Rock Club,Shopping Mall,Stadium,Thrift / Vintage Store,Trail,Speakeasy,Track Stadium,Food,Falafel Restaurant,Hotel,Beer Store,Mediterranean Restaurant,Racetrack,Video Game Store,Cupcake Shop,American Restaurant,Event Space,Gaming Cafe,Latin American Restaurant,Convenience Store,German Restaurant,Café,Argentinian Restaurant,Bakery,Salad Place,Miscellaneous Shop,Exhibit,Market,Monument / Landmark,Indian Restaurant,Food & Drink Shop,Juice Bar,Record Shop,Bagel Shop,Tea Room,Sushi Restaurant,Ramen Restaurant,Tennis Court,Beach Bar,Tennis Stadium,Boat or Ferry,Pharmacy,Cosmetics Shop,Creperie,Gym,Hostel,Plaza,Wine Bar,Art Gallery,Cocktail Bar,Mexican Restaurant,Bike Rental / Bike Share,Metro Station,Modern Greek Restaurant,Soccer Stadium,Paella Restaurant,French Restaurant,Frozen Yogurt Shop,Pedestrian Plaza,Korean Restaurant,Tourist Information Center,Burger Joint,Lingerie Store,Halal Restaurant,Gym Pool,Casino,Concert Hall,Spa,Steakhouse,Diner,Breakfast Spot,Ice Cream Shop,Pie Shop,Sandwich Place,Science Museum,Lounge,Soup Place,Gastropub,Nightclub,Bowling Alley,Bar,Train Station,Athletics & Sports,Park,Vegetarian / Vegan Restaurant,Sports Bar,Forest,Chinese Restaurant
0,AIORA,47.11230,12,39.46619,-0.34449,728444.62707,4371878.35507,0.00000,0.00000,0.00000,0.10000,0.00000,0.05000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.06000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.07000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.01000,0.00000,0.00000,0.03000,0.00000,0.00000,0.00000,0.00000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05000,0.00000,0.02000,0.00000,0.00000,0.00000,0.00000,0.01000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.01000,0.00000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.010